# import

In [23]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
tf.__version__

'2.3.1'

# Checkpoint Function

In [24]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & Preprocessing Function

In [25]:
def load_mnist():
    (train_data,train_labels), (test_data,test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data,axis=-1) # [N,28,28] -> [N,28,28,1] : [batch_size,height,width,channel]
    test_data = np.expand_dims(test_data,axis=-1)   # [N,28,28] -> [N,28,28,1] axis=위치(-1=맨끝)
    
    train_data, test_data = normalize(train_data, test_data) # [0~255] -> [0~1]
    
    train_labels = to_categorical(train_labels,10) # [N,] -> [N,10]
    test_labels = to_categorical(test_labels,10) # [N,] -> [N,10] : One hot Encoding
    return train_data, train_labels, test_data, test_labels

def normalize(train_data,test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

# Performance Function

In [26]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred = logits, y_true = labels,
                                                                   from_logits = True))
    return loss

def accuracy_fn(model, images, labels): # 정확도
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits,-1),tf.argmax(labels,-1))
    accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
    return accuracy

def grad(model, images, labels): # gradient
    with tf.GradientTape() as tape:
        loss = loss_fn(model,images,labels)
    return tape.gradient(loss,model.variables)

# Model Function

In [27]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init): # fully connected layer
    return tf.keras.layers.Dense(units=label_dim,use_bias=True,kernel_initializer=weight_init)
# unit = output으로 나갈 Channel이 몇개인가?
# use_bias = bias를 사용하는가 사용하지 않는가?
# kernel_initializer = weight 초기값. 

def sigmoid():
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

# Create model(class version)

In [28]:
class create_model_class(tf.keras.Model):
    def __init__(self,label_dim): # label_dim : Network의 Logit을 구할 때, 최종적으로 몇개의 Output을 낼 것인가?(여기선 10)
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.RandomNormal() # Random Normal Initialization
        
        self.model = tf.keras.Sequential() # 층층이 쌓아나가는 것을 List처럼.
        self.model.add(flatten()) # [N,28,28,1] -> [N,784]
        # [N,784]로 바꿔주는 것은 나중에 fully connected layer를 이용할 것이기 때문. 
        # fully connected가 아니라
        
        for i in range(2):
            self.model.add(dense(256,weight_init))
            self.model.add(sigmoid())
        
        self.model.add(dense(label_dim,weight_init))
        
    def call(self,x,training=None,mask=None):
        x = self.model(x)
        return(x)

# Create model(function version)

In [29]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256,weight_init))
        model.add(sigmoid())
        
    model.add(dense(label_dim,weight_init))
    return model

# Define data & hyper-parameter

In [30]:
"""dataset"""
train_x, train_y, test_x, test_y = load_mnist()

"""parameters"""
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

"""Graph input using Dataset API"""
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = batch_size).\
    batch(batch_size,drop_remainder=True) # batch를 몇개만큼 네트워크에 던져 줄 것인가?
    # shuffle : 데이터셋을 섞는다. input 데이터보다 크게 설정하면 Random으로 shuffle한다.
    # prefetch : batch_size만큼 학습을 하고 있을 때, 미리 메모리에 batch_size만큼 올려놓으면 학습을 좀 더 빠르게 할 수 있다.
    # batch : batch를 몇개만큼 네트워크에 던져 줄 것인가?

test_dataset = tf.data.Dataset.from_tensor_slices((test_x,test_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = len(test_x)).\
    batch(len(test_x))

In [31]:
"""model"""
network = create_model_function(label_dim)

"""training"""
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

"""writer"""
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

# check point의 역할?
# network가 학습을 하다가 끊겼을 때, 다시 재학습을 하고 싶은 경우, 변경되었던 Weight를 불러내는 역할.
# 혹은 Test 이미지에 대해서 정확도를 보고 싶은 경우에도 활용할 수 있음.
# 학습되어 저장된 Weight를 불러오는것.
checkpoint_dir = os.path.join(checkpoint_dir,model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir,model_dir)
logs_dir = os.path.join(logs_dir,model_dir)

# Restore Check point & Start train or test phase

In [32]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exists
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print('[*] Load Success')
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print('[!] Load Failed...')
    
    # train phase
    with summary_writer.as_default(): # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx,(train_input,train_label) in enumerate(train_dataset):
                grads = grad(network,train_input,train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads,network.variables))
                
                train_loss = loss_fn(network,train_input,train_label)
                train_accuracy = accuracy_fn(network,train_input,train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_fn(network,test_input,test_label)
                
                tf.summary.scalar(name='train_loss',data=train_loss,step=counter)
                tf.summary.scalar(name='train_accuracy',data=train_accuracy,step=counter)
                tf.summary.scalar(name='test_accuracy',data=test_accuracy,step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_accuracy: %.4f"\
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))

# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
[!] Load Failed...
Epoch: [ 0] [    0/  468] time: 0.5256, train_loss: 2.27871895, train_accuracy: 0.1484, test_accuracy: 0.1131
Epoch: [ 0] [    1/  468] time: 0.6592, train_loss: 2.27123785, train_accuracy: 0.1484, test_accuracy: 0.0958
Epoch: [ 0] [    2/  468] time: 0.7992, train_loss: 2.29945374, train_accuracy: 0.0859, test_accuracy: 0.0958
Epoch: [ 0] [    3/  468] time: 0.9362, train_loss: 2.28469515, train_accuracy: 0.2188, test_accuracy: 0.1932
Epoch: [ 0] [    4/  468] time: 1.0899, train_loss: 2.27140665, train_accuracy: 0.1094, test_accuracy: 0.1009
Epoch: [ 0] [    5/  468] time: 1.2285, train_loss: 2.29531431, train_accuracy: 0.0781, test_accuracy: 0.1290
Epoch: [ 0] [    6/  468] time: 1.3612, train_loss: 2.23871946, train_accuracy: 0.2031, test_accuracy: 0.1847
Epoch: [ 0] [    7/  468] time: 1.5217, train_loss: 2.22935772, train_accuracy: 0.1172, test_accuracy: 0.0997
Epoch: [ 0] [    8/  468] time: 1.6644, 

Epoch: [ 0] [   74/  468] time: 10.3396, train_loss: 1.00090158, train_accuracy: 0.7422, test_accuracy: 0.7982
Epoch: [ 0] [   75/  468] time: 10.4532, train_loss: 0.90460038, train_accuracy: 0.8281, test_accuracy: 0.7915
Epoch: [ 0] [   76/  468] time: 10.6038, train_loss: 0.95540583, train_accuracy: 0.7656, test_accuracy: 0.7805
Epoch: [ 0] [   77/  468] time: 10.7614, train_loss: 0.96442795, train_accuracy: 0.7578, test_accuracy: 0.7744
Epoch: [ 0] [   78/  468] time: 10.9040, train_loss: 0.88050902, train_accuracy: 0.7734, test_accuracy: 0.7733
Epoch: [ 0] [   79/  468] time: 11.0506, train_loss: 0.93889815, train_accuracy: 0.7344, test_accuracy: 0.7769
Epoch: [ 0] [   80/  468] time: 11.1888, train_loss: 0.97497630, train_accuracy: 0.7266, test_accuracy: 0.7836
Epoch: [ 0] [   81/  468] time: 11.3275, train_loss: 0.90800428, train_accuracy: 0.7578, test_accuracy: 0.7916
Epoch: [ 0] [   82/  468] time: 11.4554, train_loss: 0.90121031, train_accuracy: 0.7969, test_accuracy: 0.7965
E

Epoch: [ 0] [  148/  468] time: 21.3620, train_loss: 0.47643173, train_accuracy: 0.8672, test_accuracy: 0.8799
Epoch: [ 0] [  149/  468] time: 21.5395, train_loss: 0.45101595, train_accuracy: 0.8750, test_accuracy: 0.8813
Epoch: [ 0] [  150/  468] time: 21.7609, train_loss: 0.49980414, train_accuracy: 0.8750, test_accuracy: 0.8819
Epoch: [ 0] [  151/  468] time: 21.9584, train_loss: 0.62775666, train_accuracy: 0.8516, test_accuracy: 0.8836
Epoch: [ 0] [  152/  468] time: 22.1279, train_loss: 0.41206154, train_accuracy: 0.9141, test_accuracy: 0.8854
Epoch: [ 0] [  153/  468] time: 22.3015, train_loss: 0.46695015, train_accuracy: 0.8672, test_accuracy: 0.8861
Epoch: [ 0] [  154/  468] time: 22.4940, train_loss: 0.44322366, train_accuracy: 0.8906, test_accuracy: 0.8868
Epoch: [ 0] [  155/  468] time: 22.6984, train_loss: 0.43928778, train_accuracy: 0.8984, test_accuracy: 0.8861
Epoch: [ 0] [  156/  468] time: 22.8769, train_loss: 0.55924082, train_accuracy: 0.8828, test_accuracy: 0.8862
E

Epoch: [ 0] [  222/  468] time: 34.1927, train_loss: 0.28920734, train_accuracy: 0.9297, test_accuracy: 0.9021
Epoch: [ 0] [  223/  468] time: 34.3583, train_loss: 0.40806442, train_accuracy: 0.8672, test_accuracy: 0.9029
Epoch: [ 0] [  224/  468] time: 34.5158, train_loss: 0.35794890, train_accuracy: 0.9062, test_accuracy: 0.9049
Epoch: [ 0] [  225/  468] time: 34.6684, train_loss: 0.37608722, train_accuracy: 0.8828, test_accuracy: 0.9047
Epoch: [ 0] [  226/  468] time: 34.8141, train_loss: 0.31358886, train_accuracy: 0.9297, test_accuracy: 0.9040
Epoch: [ 0] [  227/  468] time: 34.9694, train_loss: 0.36068785, train_accuracy: 0.8984, test_accuracy: 0.9037
Epoch: [ 0] [  228/  468] time: 35.1206, train_loss: 0.32591635, train_accuracy: 0.8906, test_accuracy: 0.9045
Epoch: [ 0] [  229/  468] time: 35.2672, train_loss: 0.35735166, train_accuracy: 0.9219, test_accuracy: 0.9048
Epoch: [ 0] [  230/  468] time: 35.4194, train_loss: 0.27090079, train_accuracy: 0.9453, test_accuracy: 0.9039
E

Epoch: [ 0] [  295/  468] time: 46.2849, train_loss: 0.25973409, train_accuracy: 0.9297, test_accuracy: 0.9141
Epoch: [ 0] [  296/  468] time: 46.4696, train_loss: 0.36958617, train_accuracy: 0.9141, test_accuracy: 0.9148
Epoch: [ 0] [  297/  468] time: 46.6441, train_loss: 0.26511610, train_accuracy: 0.9062, test_accuracy: 0.9143
Epoch: [ 0] [  298/  468] time: 46.8167, train_loss: 0.24842255, train_accuracy: 0.9297, test_accuracy: 0.9140
Epoch: [ 0] [  299/  468] time: 46.9783, train_loss: 0.25387648, train_accuracy: 0.9531, test_accuracy: 0.9139
Epoch: [ 0] [  300/  468] time: 47.1727, train_loss: 0.30735222, train_accuracy: 0.9141, test_accuracy: 0.9130
Epoch: [ 0] [  301/  468] time: 47.3443, train_loss: 0.41136599, train_accuracy: 0.8906, test_accuracy: 0.9124
Epoch: [ 0] [  302/  468] time: 47.5098, train_loss: 0.24331391, train_accuracy: 0.9453, test_accuracy: 0.9125
Epoch: [ 0] [  303/  468] time: 47.6684, train_loss: 0.31804100, train_accuracy: 0.8984, test_accuracy: 0.9130
E

Epoch: [ 0] [  369/  468] time: 58.0786, train_loss: 0.22254059, train_accuracy: 0.9375, test_accuracy: 0.9151
Epoch: [ 0] [  370/  468] time: 58.2101, train_loss: 0.39487931, train_accuracy: 0.8828, test_accuracy: 0.9144
Epoch: [ 0] [  371/  468] time: 58.3509, train_loss: 0.23374408, train_accuracy: 0.9375, test_accuracy: 0.9139
Epoch: [ 0] [  372/  468] time: 58.4815, train_loss: 0.27117085, train_accuracy: 0.9531, test_accuracy: 0.9137
Epoch: [ 0] [  373/  468] time: 58.6182, train_loss: 0.30670184, train_accuracy: 0.9141, test_accuracy: 0.9135
Epoch: [ 0] [  374/  468] time: 58.7522, train_loss: 0.23626755, train_accuracy: 0.9453, test_accuracy: 0.9144
Epoch: [ 0] [  375/  468] time: 58.8878, train_loss: 0.30234239, train_accuracy: 0.9062, test_accuracy: 0.9160
Epoch: [ 0] [  376/  468] time: 59.0135, train_loss: 0.31649876, train_accuracy: 0.9297, test_accuracy: 0.9168
Epoch: [ 0] [  377/  468] time: 59.1423, train_loss: 0.24074145, train_accuracy: 0.9219, test_accuracy: 0.9177
E

Epoch: [ 0] [  443/  468] time: 68.2910, train_loss: 0.19802792, train_accuracy: 0.9453, test_accuracy: 0.9251
Epoch: [ 0] [  444/  468] time: 68.4326, train_loss: 0.22276773, train_accuracy: 0.9297, test_accuracy: 0.9254
Epoch: [ 0] [  445/  468] time: 68.5723, train_loss: 0.20425910, train_accuracy: 0.9375, test_accuracy: 0.9253
Epoch: [ 0] [  446/  468] time: 68.7009, train_loss: 0.32955939, train_accuracy: 0.9219, test_accuracy: 0.9249
Epoch: [ 0] [  447/  468] time: 68.8380, train_loss: 0.24796951, train_accuracy: 0.9297, test_accuracy: 0.9247
Epoch: [ 0] [  448/  468] time: 68.9676, train_loss: 0.34209463, train_accuracy: 0.8984, test_accuracy: 0.9249
Epoch: [ 0] [  449/  468] time: 69.0953, train_loss: 0.32876778, train_accuracy: 0.9141, test_accuracy: 0.9245
Epoch: [ 0] [  450/  468] time: 69.2240, train_loss: 0.27551842, train_accuracy: 0.9062, test_accuracy: 0.9245
Epoch: [ 0] [  451/  468] time: 69.3506, train_loss: 0.32085970, train_accuracy: 0.8906, test_accuracy: 0.9241
E

In [33]:
# test acc = 92.41